In [2]:
%load_ext autoreload
%autoreload 2

import torch
from torch_geometric.data import Data

data_list = torch.load("../../data/train_data/edge_features2/datalist_batch_1.pt")
print(data_list[0])                     
print(len(data_list))

# https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.data.Data.html
print(type(data_list[0]))

Data(edge_index=[2, 31635], num_nodes=31635, x=[31635, 6], pos=[31635, 3, 2], y=[31635, 1], edge_is_directed=[31635], mode_stats_diff=[6, 3], mode_stats_diff_perc=[6, 3])
10
<class 'torch_geometric.data.data.Data'>


In [4]:
import os
import sys
import torch

# Add the 'scripts' directory to Python Path
scripts_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

from gnn.models import EIGNLaplacianConv

model = EIGNLaplacianConv(
    in_channels_signed=6,
    out_channels_signed=1,
    in_channels_unsigned=6,
    out_channels_unsigned=1,
    hidden_channels_signed=32,
    hidden_channels_unsigned=32,
    dtype=torch.float,
    num_blocks=4,
).eval()

with torch.no_grad():
    outputs = model(
        x_signed=data_list[0].x,
        x_unsigned=None,
        edge_index=data_list[0].edge_index,
        is_directed=data_list[0].edge_is_directed,
    )
outputs.signed, outputs.unsigned

torch.Size([31635, 6])
torch.Size([31635, 32])
torch.Size([31635, 32])
torch.Size([31635, 32])
torch.Size([31635, 32])
torch.Size([31635, 32])
torch.Size([31635, 32])


(tensor([[ 0.0877],
         [ 0.5377],
         [-0.5359],
         ...,
         [-0.0551],
         [ 0.3967],
         [ 0.0349]]),
 tensor([[ 0.3596],
         [-2.1780],
         [-2.1242],
         ...,
         [ 1.7848],
         [ 0.2460],
         [ 0.7414]]))


Parameters:
    
    x (torch.Tensor, optional) – Node feature matrix with shape [num_nodes, num_node_features]. (default: None)

    edge_index (LongTensor, optional) – Graph connectivity in COO format with shape [2, num_edges]. (default: None)<>

    edge_attr (torch.Tensor, optional) – Edge feature matrix with shape [num_edges, num_edge_features]. (default: None)

    y (torch.Tensor, optional) – Graph-level or node-level ground-truth labels with arbitrary shape. (default: None)

    pos (torch.Tensor, optional) – Node position matrix with shape [num_nodes, num_dimensions]. (default: None)

    time (torch.Tensor, optional) – The timestamps for each event with shape [num_edges] or [num_nodes]. (default: None)


PyTorch Geometric (PyG) data batches are saved in the `result_path` as `.pt` tensor files. Each sample is a homogenous graph (representing a scenario as decribed above) with the following attributes per node (road segment):
- `x`: Node features:
    - Volume Base Case
    - Capacity Base Case
    - Capacity Reduction
    - Maximum Speed
    - Road Type
    - Length
    - And additionally, if `use_allowed_modes` is `True`, then booleans indicating whether `Car`, `Bus`, `Public Transport`, `Train`, `Rail`, and `Subway` are allowed on the road segment.
- `y`: Target for the GNN, difference in traffic volume between the base case and the simulation run (with policy applied).
- `pos`: x and y coordinates of the start, middle, and end of the road segment.

And the following attributes for the entire graph:
- `edge_index`: Edges of the graph, defined by the start and end nodes of each edge.
- `mode_stats_diff`: Difference in travel mode statistics between the base case and the simulation run (with policy applied).
- `mode_stats_diff_per`: `mode_stats_diff` in percentage (compared to the base case).

In [ ]:
import torch

# Load the data
data_list = torch.load("../../data/test_data/datalist_batch_1.pt")
sample = data_list[0]  # Let's examine the first graph

# Extract edge_index and node features
edge_index = sample.edge_index
node_features = sample.x

# 1. Compute edge features
# Orientation-equivariant features (e.g., traffic volume difference)
edge_features_equ = node_features[edge_index[0]] - node_features[edge_index[1]]

# Orientation-invariant features (e.g., capacity, speed limit, etc.)
edge_features_inv = (node_features[edge_index[0]] + node_features[edge_index[1]]) / 2

# 2. Structure edge features into Xequ and Xinv
Xequ = edge_features_equ  # Directional edge features
Xinv = edge_features_inv  # Non-directional edge features

# 3. Handle directed and undirected edges
# Aggregate bidirectional edges (u, v) and (v, u) into a single undirected edge
directed_edges = edge_index.T.tolist()
undirected_edges = set(tuple(sorted(edge)) for edge in directed_edges)
undirected_edge_index = torch.tensor(list(undirected_edges)).T

# 4. Prepare the transformed data
transformed_data = {
    "edge_index": undirected_edge_index,  # Updated edge index
    "Xequ": Xequ,  # Orientation-equivariant edge features
    "Xinv": Xinv,  # Orientation-invariant edge features
    "y_edge": sample.y  # Edge-level target (if applicable)
}

# Print the transformed data for verification
print("=== Transformed Data ===")
print(f"Edge Index:\n{transformed_data['edge_index']}")
print(f"Orientation-Equivariant Features (Xequ):\n{transformed_data['Xequ']}")
print(f"Orientation-Invariant Features (Xinv):\n{transformed_data['Xinv']}")